# Entrenamiento del Discriminador.

Este notebook entrena un clasificador binario basado en RoBERTa para distinguir textos
estilo Shakespeare (positivos) de textos no Shakespeare (negativos).

Los negativos incluyen tanto controles gruesos (otros géneros/autores)
como controles finos (textos literarios más cercanos en estilo).

El modelo entrenado se utiliza luego para obtener una métrica de qué tan eficiente fue el finetune del modelo Mistral 7B.

## 0. Dependencias.

In [ ]:
!pip install -q transformers datasets accelerate

## 1. Imports y rutas.

In [ ]:
import os
from datasets import Dataset

# rutas
POS_DIR = "/content/drive/MyDrive/StoryWriter/Data/Training_data/clasificator_train/positivos"
NEG_DIR = "/content/drive/MyDrive/StoryWriter/Data/Training_data/clasificator_train/negativos_gruesos"
NEGF_DIR = "/content/drive/MyDrive/StoryWriter/Data/Training_data/clasificator_train/negativos_finos"

## 3. Carga de textos.

In [ ]:
def cargar_textos(carpeta, etiqueta):
    """
    Carga archivos .txt de una carpeta y asigna una etiqueta binaria.
    etiqueta = 1 -> positivo (Shakespeare-like)
    etiqueta = 0 -> negativo (no Shakespeare)
    """
    ejemplos = []
    for fname in os.listdir(carpeta):
        if fname.endswith(".txt"):
            ruta = os.path.join(carpeta, fname)
            with open(ruta, "r", encoding="utf-8") as f:
                texto = f.read()
            ejemplos.append({"text": texto, "label": etiqueta})
    return ejemplos

positivos = cargar_textos(POS_DIR, 1)
negativos_gruesos = cargar_textos(NEG_DIR, 0)
negativos_finos = cargar_textos(NEGF_DIR, 0)

# balance simple: usar la misma cantidad de negativos gruesos que finos
negativos_gruesos = negativos_gruesos[len(negativos_finos):]
negativos = negativos_gruesos + negativos_finos

datos = positivos + negativos

print("Total de ejemplos de entrenamiento:", len(datos))

## 4. Construcción del dataset.

In [ ]:
dataset = Dataset.from_list(datos)
dataset

## 5. Tokenización.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

MAX_LENGTH = 512

def tokenize(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )

tokenized = dataset.map(tokenize, batched=True)
tokenized = tokenized.remove_columns(["text"])
tokenized.set_format("torch")

# debug en CUDA (para Colab)
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

## 6. Modelo y configuración del entrenamiento.

In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)

model = AutoModelForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=2
)

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/StoryWriter/Clasificador/Grueso+fino",
    learning_rate=2e-5,
    num_train_epochs=4,
    per_device_train_batch_size=8,
    weight_decay=0.01,
    logging_steps=100,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    tokenizer=tokenizer,
)

## 7. Entrenamiento.

In [ ]:
trainer.train()

## 8. Guardado.

In [ ]:
save_dir = "/content/drive/MyDrive/StoryWriter/Clasificador/Grueso+fino"

trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

print("Modelo guardado en:", save_dir)